In [5]:
from modules.tweet_data import read_raw_data
from modules.spacy import spacy_twitter_model

In [2]:
tweet_df = read_raw_data()

Reading data


In [ ]:
REGEX_DICT = {'link': "http\S+",
              'piclink': "pic.twitter.com\S+",
              'hashtag': "\B#\w*[a-zA-Z]+\w*|#\w*[a-zA-Z]+\w*",
              'email': "[a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+",
              'add': "\s([@][\w_-]+)|[@][\w_-]+"}

for key, value in REGEX_DICT.items():
    print('Filtering', key)
    pearkes.tweet = pearkes.tweet.apply(lambda tweet: re.sub(value, key.upper(), tweet))

In [3]:
pearkes = tweet_df[tweet_df.handle=='pearkes']

In [11]:
with open('pearkes.txt', 'w') as filehandle:
    for listitem in pearkes.tweet.to_list():
        filehandle.write('%s\n' % listitem)

In [15]:
!python run_language_modeling.py \
    --output_dir='here/' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --train_data_file='pearkes.txt' \
    --do_eval \
    --eval_data_file='pearkes.txt'

07/28/2020 12:39:38 - INFO - transformers.training_args -   PyTorch: setting up devices
07/28/2020 12:39:38 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
07/28/2020 12:39:38 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='here/', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jul28_12-39-38_Eduardos-MacBook-Pro.local', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level='O1', local_rank=-1, tpu_num_cores=None, tpu_metrics_debug=False, debug=Fal

In [207]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

%%time
tokenizer.pad_token = tokenizer.eos_token
data = tokenizer.batch_encode_plus(pearkes.tweet, 
                            return_tensors="pt",
                            padding=True,truncation=True)

from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)

from transformers import TextDataset, LineByLineTextDataset
a = LineByLineTextDataset(tokenizer=tokenizer, file_path='listfile.txt', block_size=1)

from torch.utils.data.dataset import Dataset

# define list of places
places = pearkes.tweet

with open('listfile.txt', 'w') as filehandle:
    for listitem in places:
        filehandle.write('%s\n' % listitem)
        

class GPReviewDataset(Dataset):
    def __init__(self, reviews):
        self.reviews = reviews
    def __len__(self):
        return data['input_ids'].shape[0]
    def __getitem__(self, item):

        return {
          'input_ids': self.reviews['input_ids'][item].flatten(),
          'attention_mask': self.reviews['attention_mask'][item].flatten(),
        }
    

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./pearkes",
    overwrite_output_dir=True,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=a,
    prediction_loss_only=True,
)

trainer.train()

Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
